In [1]:
# Auth Headers

In [2]:


import requests
from config import CLIENT_ID, SECRET_KEY, OPENAI_KEY
from config import reddit_auth_data as data

auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

headers = {
    'User-Agent': 'RedditAutomod/0.0.1'
}

res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth,
                    data=data,
                    headers=headers)
TOKEN = res.json()['access_token']

headers['Authorization'] = f'bearer {TOKEN}'

In [3]:
subreddit = 'r/explainlikeimfive'

In [ ]:
# Get subreddit rules
resp = requests.get(f'https://oauth.reddit.com/{subreddit}/about/rules', headers=headers).json()
# print(resp)
rulesPrompt = ""

for ix,el in enumerate(resp['rules']):
    
    ruleDesc = (el['description'])
    singleRuleTemplate = f'Rule {ix}) {ruleDesc}\n\n'
    rulesPrompt += singleRuleTemplate

print(rulesPrompt)

In [7]:
# Get good posts
resp = requests.get(f'https://oauth.reddit.com/{subreddit}/hot?limit=25', headers=headers).json()
for el in resp['data']['children']:
    print(f'Title: {el["data"]["title"]} \nPost Text: {el["data"]["selftext"]}\n')

Title: ELI5: why can’t the NFL just put a little tracker in the football so there’s no guessing on a yardage gained/ 1st down/ touch down/ out of bounds play? 
Post Text: Just started watching football with my SO in the last few years, I don’t understand why this isn’t a thing? Seems like it would get rid of a lot of confusion

Title: ELI5: Why does taking a liver shot cause body to shutdown? 
Post Text: Ryan Garcia vs Gervonta Davis comes into mind. Garcia couldn't stand after taking one shot to the liver. Same goes for UFC fights... Brock Lesnar against Overeem, no one seems to take a clean liver shot. What exactly happens when your body takes a liver shot? Why is it different than taking shot at a different body part?

Title: ELI5: Why do RAM modules always come in powers of 2? 
Post Text: Unless the RAM is 1GB, it's almost always going to be in powers of 2.

2G

4GB

8G

16GB

Hell, even 1GB could still technically be considered a power of 2... if the exponent is 0.

It's virtually

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key=OPENAI_KEY)
             
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.chains import LLMChain

template = "Here are the rules for the subreddit {subreddit}:\n {rules}"
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chain = LLMChain(llm=llm, prompt=chat_prompt, output_parser=StrOutputParser())
out = chain.run(subreddit=subreddit,rules=rulesPrompt,text="Create a post for this subreddit that would violate rule 4")
print(out)

Limitations:
would need to recursively follow links and only operate on text of webpages - parsing webpages is not simple. 
